# !IMPORTANTE: 
    debido a que muchos datos no se encuentran en condiciones de ser graficados se contara  con un  segundo archivo EDA_Parte2 donde se realizara tanto el analisis estadistico como el analisis de correlacion bi y multivariado

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import ast
from matplotlib import pyplot as plt
import ETL_functions as etf

<hr>

## EDA "steam_games"

<hr>

### cargando datos y primeras observaciones

In [2]:
df_sg = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json", lines=True)#cargando el archivo

In [3]:
df_sg.shape # dimensiones (filas,columnas) 

(120445, 13)

In [4]:
df_sg.sample(5)#ejecute esta linea aprox(5) veces para tener una vision mas amplia de algunos datos aleatorios del df 
               #notando que hay muchas filas llenas de valores None o NaN
               #a demas hay columnas que contienen listas como valores

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
36460,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
68670,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
113682,None,"[Casual, Simulation]",Rocksmith® 2014 – Kiss - “Love Gun”,Rocksmith® 2014 – Kiss - “Love Gun”,http://store.steampowered.com/app/390366/Rocks...,2015-10-13,"[Casual, Simulation]",http://steamcommunity.com/app/390366/reviews/?...,"[Single-player, Shared/Split Screen, Downloada...",2.99,0.0,390366.0,Ubisoft - San Francisco
29050,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
91241,Dovetail Games - Trains,[Simulation],Train Simulator: GWR Small Prairies Loco Add-On,Train Simulator: GWR Small Prairies Loco Add-On,http://store.steampowered.com/app/277753/Train...,2014-12-10,"[Simulation, Trains, Building, Driving, Open W...",http://steamcommunity.com/app/277753/reviews/?...,"[Single-player, Downloadable Content, Steam Ac...",24.99,0.0,277753.0,Victory Works


### tipos de datos

In [5]:
df_sg.info()#podemos notar que solo contamos con 2 columnas numericas (id,early_access) siendo inecesario analizar con un describe estas
            #podemos ver ademas que early access es float cuando deberia ser un bool(true o false){aclarado en el diccionario} o int (0 o 1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [6]:
# podemos notar que la columna price no se encuentra en float esto puede deberse a que hay valores no convertibles
# para verificar esto e creado una funcion que me ayudara cuando deba corregir esto en un archivo llamado ETL_functions.py el cual importe como etf
etf.valores_no_numericos(df_sg["price"])

['Free To Play',
 'Free to Play',
 'Free',
 'Free Demo',
 'Play for Free!',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Free Mod',
 'Install Theme',
 'Third-party',
 'Play Now',
 'Free HITMAN™ Holiday Pack',
 'Play the Demo',
 'Starting at $499.00',
 'Starting at $449.00',
 'Free to Try',
 'Free Movie',
 'Free to Use']

### valores nulos

In [7]:
df_sg.isna().sum() #estos son los campos con valores nulos y su cantidad de nulos

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

### valores duplicados

In [8]:
#tuve que cambiar el tipo de datos a str por que tenian listas como valor  y no me permitian validar duplicados
df_sg["genres"] = df_sg["genres"].astype(str)
df_sg["tags"] = df_sg["tags"].astype(str)
df_sg["specs"] = df_sg["specs"].astype(str)

In [9]:
df_sg.duplicated().sum()# cantidad de registros duplicados

88309

In [10]:
120445 - 88309 # filas totales - filas duplicadas = aproximacion de (datos a trabajar)
               # tener en cuenta que al borrar duplicados aun asi nos quedara un registro
               # unico de las filas que tenian todos sus valores en null la cual deberemos retirar

32136

<hr>

## EDA "user_reviews"

### cargando datos y primeras observaciones

In [11]:
#carga de el archivo
reviews = []
with open(r"../Datasets/user_reviews.json/australian_user_reviews.json", 'r', encoding='utf-8') as f:
    # loop for insert data into list
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
df_ur = pd.DataFrame(reviews)

In [12]:
df_ur.sample(5)# repetimos el sample varias veces
                # notamos que la columna reviews contiene listas
                # y que user id contiene tanto numeros como cadenas (cosa que puede que este bien por lo que no vamos a profundizar)

,user_id,user_url,reviews
3912,76561198067285046,http://steamcommunity.com/profiles/76561198067...,"[{'funny': '', 'posted': 'Posted December 4, 2..."
17983,76561198028024146,http://steamcommunity.com/profiles/76561198028...,"[{'funny': '', 'posted': 'Posted March 13.', '..."
4877,76561198046976160,http://steamcommunity.com/profiles/76561198046...,"[{'funny': '', 'posted': 'Posted September 21,..."
8828,27601MCM,http://steamcommunity.com/id/27601MCM,"[{'funny': '', 'posted': 'Posted September 12,..."
13301,2spookis,http://steamcommunity.com/id/2spookis,"[{'funny': '', 'posted': 'Posted November 27, ..."


### tipos de datos
|columna |tipo|
|--------|----|
|user_id | str|
|user_url| str|
|reviews |list|

### valores nulos

In [13]:
df_ur.isna().sum() #estos son los campos con valores nulos y su cantidad de nulos  al parecer no hay valores nulos pero podriaa haber reviews que contengan listas vacias

user_id     0
user_url    0
reviews     0
dtype: int64

### valores duplicados

In [14]:
df_ur["reviews"] = df_ur["reviews"].astype(str) #paso la columna a str para poder contar los registros dupliados

In [15]:
df_ur.duplicated().sum() # hay 313 registros duplicados lo que no nos interesa y los borraremos en el ETL

313

<hr>

## EDA "users_items"

### cargando datos y primeras observaciones

In [16]:
items = []

with open(r'../Datasets/users_items.json/australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        items.append(ast.literal_eval(line))

df_ui = pd.DataFrame(items)

In [17]:
df_ui.sample(5)#repetimos el sample varias veces

,user_id,items_count,steam_id,user_url,items
45492,Greggs_place,8,76561198005852745,http://steamcommunity.com/id/Greggs_place,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
28063,gtr33m,136,76561198006147691,http://steamcommunity.com/id/gtr33m,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
2959,76561198070410552,101,76561198070410552,http://steamcommunity.com/profiles/76561198070...,"[{'item_id': '240', 'item_name': 'Counter-Stri..."
79401,76561198090065255,6,76561198090065255,http://steamcommunity.com/profiles/76561198090...,"[{'item_id': '205790', 'item_name': 'Dota 2 Te..."
53242,76561198048004455,0,76561198048004455,http://steamcommunity.com/profiles/76561198048...,[]


## tipos de datos

|columna|tipo|
|-------|----|
|user_id|str|
|item_counts|int|
|steam_id|int|
|user_url|str|
|item|list|

In [18]:
df_ui.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [19]:
#steam_id no se encuentra en int pero al parecer no tiene valores no numericos solo se debe a que tiene numeros muy grandes y el tipo de dato de pandas debe ser un dtype que pueda contener  un numero asi de grande 
etf.valores_no_numericos(df_ui["steam_id"])

[]

### valores nulos

In [20]:
df_ui.isna().sum() #estos son los campos con valores nulos y su cantidad de nulos 
#dice no tener valores nulos pero items puede contener listas vacias (como  podemos verificar en el sample y son equivalentes a nulls)

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

### valores duplicados

In [21]:
df_ui["items"] = df_ui["items"].astype(str)#convertirmos a str para poder verificar duplicados

In [22]:
df_ui.duplicated().sum()#tiene 657 que los borrare

657

<hr>

## vizualizando filas para hallar relaciones

In [23]:
df_sg.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [24]:
df_ur.head(1)

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."


In [25]:
df_ui.head(1)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."


### vizualizando columnas anidadadas en listas

In [26]:
df_ui["items"][0]

'[{\'item_id\': \'10\', \'item_name\': \'Counter-Strike\', \'playtime_forever\': 6, \'playtime_2weeks\': 0}, {\'item_id\': \'20\', \'item_name\': \'Team Fortress Classic\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'30\', \'item_name\': \'Day of Defeat\', \'playtime_forever\': 7, \'playtime_2weeks\': 0}, {\'item_id\': \'40\', \'item_name\': \'Deathmatch Classic\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'50\', \'item_name\': \'Half-Life: Opposing Force\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'60\', \'item_name\': \'Ricochet\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'70\', \'item_name\': \'Half-Life\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'130\', \'item_name\': \'Half-Life: Blue Shift\', \'playtime_forever\': 0, \'playtime_2weeks\': 0}, {\'item_id\': \'300\', \'item_name\': \'Day of Defeat: Source\', \'playtime_forever\': 4733, \'playtime_2weeks\': 0}, {\'ite

In [27]:
df_ur["reviews"][0]

'[{\'funny\': \'\', \'posted\': \'Posted November 5, 2011.\', \'last_edited\': \'\', \'item_id\': \'1250\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': \'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.\'}, {\'funny\': \'\', \'posted\': \'Posted July 15, 2011.\', \'last_edited\': \'\', \'item_id\': \'22200\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': "It\'s unique and worth a playthrough."}, {\'funny\': \'\', \'posted\': \'Posted April 21, 2011.\', \'last_edited\': \'\', \'item_id\': \'43110\', \'helpful\': \'No ratings yet\', \'recommend\': True, \'review\': \'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this game is definitely worth it and I hope they do a sequel...so buy the game so I get a sequel!\'}

### estableciendo relaciones

|df_ur|df_ui|                
|-----|-----|                
|user_id|user_id/steam_id|
|user_url|user_url|

user_id == steam_id por lo que se puede rellenar los valores faltantes de una con los de la otra,
a su vez se pueden rellenar con los valores de las mismas columnas en user_items y viceversa(user_reviews)

|df_sg|df_ui|
|-----|-----|
|app_name|item_name|
|id|item_id|


### df_sg

tiene como columnas que vamos a usar genero que a su ves se puede rellenar con cualquier etiqueta en comun con tags y specs

y precio columna que usaremos para alguna funcion

usaremos title como app_name donde notengamos el valor de app_name y si no contamos con ninguno de los dos valores intentaremos conseguirlo atravez de user_items

###  df_ur

contamos con user_id y user_url en caso de necesitar completar en otra lista o viceversa

en la columna reviews contamos con diferentes datos como

fecha en que se posteo al review 
item_id
review la que usaremos para el analisis de sentimiento
y si es recommendado o no el juego (esto puede decir si es positiva o negativa la reseña)

podrian borrarse 

last edited

y funny (esto suponiendo que no importa si se marca como divertido si no tiene texto la reseña)

In [28]:
generos = etf.valores_lista(df_sg,"genres")
tags = etf.valores_lista(df_sg,"tags")
specs = etf.valores_lista(df_sg,"specs")

In [29]:
elementos_comunes = set(generos) & set(tags)

In [37]:
comunes = set(tags) & set(specs)

In [36]:
comunes # entre tags y specs solo se encuentra co-op y como no tenemos ni un solo co-op en generos no lo tendre en cuenta ya que solo tomo en cuenta las columnas que se encuentran en genero
#por lo tanto borrare la columna specs y los valores ["None"] que se encuentran en todas estas columnas

{'None'}

In [30]:
elementos_comunes# estos son los valores que en caso de no tener genero si estos se encuentran en tags se transfieren a genero

{'Action',
 'Adventure',
 'Audio Production',
 'Casual',
 'Early Access',
 'Education',
 'Free to Play',
 'Indie',
 'Massively Multiplayer',
 'None',
 'Photo Editing',
 'RPG',
 'Racing',
 'Simulation',
 'Software Training',
 'Sports',
 'Strategy',
 'Utilities',
 'Video Production',
 'Web Publishing'}

In [38]:
# como conclusion de la relacion entre estas 3 columnas solo 2 estan relacionadas entre si y pueden enriquecerse mutuamente
# por lo cual utilizare los elementos comunes que se encuentren dentro de la columna tags de la misma fila para rellenarlos y luego dropear esta 


*si una fila no contiene ni genero ni tags dentro de los elementos comunes sera dropeada

tambien dropearemos early access